In [2]:
import pandas
import category_encoders as ce

df = pandas.read_csv("../DataSet/af2_dataset_training_labeled.csv")

In [3]:
# Convert feature true/false -> 1/0
binaryDataLst=['feat_A', 'feat_C', 'feat_D', 'feat_E', 'feat_F', 'feat_G', 'feat_H', 'feat_I', 'feat_K', 'feat_L', 'feat_M', 'feat_N', 'feat_P', 'feat_Q', 'feat_R',
               'feat_S', 'feat_T', 'feat_V', 'feat_W', 'feat_Y', 'feat_DSSP_H', 'feat_DSSP_B', 'feat_DSSP_E', 'feat_DSSP_G', 'feat_DSSP_I', 'feat_DSSP_T', 'feat_DSSP_S']

ram2 = df[binaryDataLst].astype('int')

In [4]:
# Convert entry to a set of range number
unit_range = df["entry"].unique()
Entry_list = df["entry"].replace(unit_range, list(range(1, len(unit_range) + 1)), inplace=False)

In [5]:
# construct min and max to normalize the already int column
minVals = {}
maxVals = {}
val_col = ['feat_PHI', 'feat_PSI', 'feat_TAU', 'feat_THETA', 'feat_BBSASA', 'feat_SCSASA', 'feat_pLDDT', 'feat_DSSP_6', 'feat_DSSP_7', 'feat_DSSP_8',
               'feat_DSSP_9', 'feat_DSSP_10', 'feat_DSSP_11', 'feat_DSSP_12', 'feat_DSSP_13', 'coord_X', 'coord_Y', 'coord_Z']
val_list = df[val_col].join(Entry_list)
val_col.append("entry")
normalized = pandas.DataFrame()
for i in val_col:
    minVals[i] = min(val_list[i])
    maxVals[i] = max(val_list[i])
    normalized[i] = (val_list[i] - minVals[i])/(maxVals[i] - minVals[i])

In [6]:
#join for output
output = ram2.join(normalized)

In [7]:
## Dealing with the entry index:
entry_indexs = df["entry_index"]
entrys = output["entry"]
index_num = {}
entry_count = 1
unque_type_ind = 0
for i in range(1, len(entry_indexs)):
    if entry_indexs[i] == 0:
        index_num[unque_type_ind] = entry_count
        entry_count = 1
        unque_type_ind = unque_type_ind + 1
        continue
    entry_count = entry_count + 1
index_num[unque_type_ind] = entry_count

j = 0
pos = 0
for i in range(0, len(entry_indexs)):
    entry_indexs[i] = entry_indexs[i] / index_num[j]
    # print(index_num[j])
    if index_num[j] == pos:
        j = j + 1
        pos = 0
    pos = pos + 1

/tmp/ipykernel_195701/642717377.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entry_indexs[i] = entry_indexs[i] / index_num[j]


In [13]:
output1 = output.join(entry_indexs)
output2 = output1.join(df["y_Ligand"].astype('int'))


0         0
1         0
2         0
3         0
4         0
         ..
497161    0
497162    0
497163    0
497164    0
497165    0
Name: y_Ligand, Length: 497166, dtype: int64

In [9]:
output2.to_csv('../DataSet/normalized.csv')